In [1]:
%%capture
!pip install sacrebleu
!pip install rouge_score

In [2]:
import pandas as pd
import nltk
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from nltk.tokenize import sent_tokenize

from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

from typing import List, Tuple
from memotion_utility import load_dataset_text_only,load_data

2024-07-30 14:54:34.634267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 14:54:34.634377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 14:54:34.773359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64
train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64
train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64


In [3]:
CSV_FILE = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
MAX_LEN = 512
LR = 2e-5
batch_size = 8
epochs = 10
#metric_name = 'f1_macro'

model_name = 'google/flan-t5-small'

downsample = True
captions = '/kaggle/input/memotion-with-captions/caption_BLIP.csv'
rationales = '/kaggle/input/memotion-rationales/rationales.csv'

In [4]:
# dfs = []

# # Read each CSV file and append to the list
# for filename in sorted(os.listdir(rationales)):
#     if filename.endswith(".csv"):
#         file_path = os.path.join(rationales, filename)
#         dfs.append(pd.read_csv(file_path))

# # Concatenate all dataframes
# combined_df = pd.concat(dfs, ignore_index=True)
# combined_df

In [5]:
dataset = load_data(CSV_FILE, downsample = True,captions = captions,rationales=rationales)

train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64


In [6]:
train,val,test = dataset
test

,image_name,text,offensive,label,caption,combined,combined_distilled,rationale
4985,image_5875.jpeg,This country is * going to hell Not enough dir...,offensive,1,this country is going to hell not enough dirty...,Given a Text: This country is * going to hell ...,Given a Text: This country is * going to hell ...,NaN
3926,image_6833.jpg,COME BACK TO AMERICA WITH YOU?,offensive,1,there is a girl that is talking to a boy that ...,"Given a Text: COME BACK TO AMERICA WITH YOU?, ...","Given a Text: COME BACK TO AMERICA WITH YOU?, ...",NaN
2493,image_2752.jpg,ONLY OUR PM KNOWS WHERE THE CAMERA IS 9th EAS...,offensive,1,there is a group of men in suits standing toge...,Given a Text: ONLY OUR PM KNOWS WHERE THE CAM...,Given a Text: ONLY OUR PM KNOWS WHERE THE CAM...,NaN
3139,image_6720.png,Reasons to livs memes Happy Pepe ifunny.co,not offensive,0,the frog is reading a book and the frog is rea...,Given a Text: Reasons to livs memes Happy Pepe...,Given a Text: Reasons to livs memes Happy Pepe...,NaN
2803,image_6879.png,IN MY COUNTRY IF YOU ENTER THE WOMEN SECTION O...,not offensive,0,there are many people standing on the subway p...,Given a Text: IN MY COUNTRY IF YOU ENTER THE W...,Given a Text: IN MY COUNTRY IF YOU ENTER THE W...,NaN
...,...,...,...,...,...,...,...,...
3156,image_331.jpg,El Arroyo Gustin RAISE YOUR HAND IF YOU STILL ...,offensive,1,there is a sign that says raise your hand if y...,Given a Text: El Arroyo Gustin RAISE YOUR HAND...,Given a Text: El Arroyo Gustin RAISE YOUR HAND...,NaN
1024,image_3318.jpg,PUTIN MUST BE REALLY LOVED VOTED BY 100% OF RU...,not offensive,0,putin must be really loved voted by 100 % of r...,Given a Text: PUTIN MUST BE REALLY LOVED VOTED...,Given a Text: PUTIN MUST BE REALLY LOVED VOTED...,NaN
4274,image_6901.jpg,I prefer the real aunt may. I said the real au...,offensive,1,there are many pictures of a woman and a man i...,Given a Text: I prefer the real aunt may. I sa...,Given a Text: I prefer the real aunt may. I sa...,NaN
2594,image_2893.jpg,Has very first Nightmare Night It gets cancell...,offensive,1,this is a picture of a picture of a dog with a...,Given a Text: Has very first Nightmare Night I...,Given a Text: Has very first Nightmare Night I...,NaN


In [7]:
dataset = load_dataset_text_only(CSV_FILE, downsample = True,captions = captions,rationales = rationales,mode = 'distil')

train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64


In [8]:
dataset['train'][0]

{'text': 'Given a Text: Grandparents and technology... TA Geez Grandma! It\'s not that hard! Go into Settings... select wi-fi... Select it! Tap it with your finger... OMG any finger!! Grrrrrrr FWD: FWD: "HA THIS IS TOTALLY ME I HOPE YOU HAVE A GRATE DAY LOVE GRAMMA", which is embedded in an Image described by this caption: there is a child that is talking on a cell phone. please provide a streamlined rationale associated with the meme',
 'label': 'The meme you provided contains a textual content that is embedded in an image, and the overall semantics of the meme are holistically presented. After analyzing the meme, I have identified several reasons why it may be considered offensive.\n\nFirstly, the use of the phrase "TA Geez Grandma" could be perceived as disrespectful and dismissive towards elderly individuals, particularly grandparents. The tone of the text is casual and flippant, which may be seen as disregarding the importance of respecting one\'s elders.\n\nSecondly, the image de

In [9]:
# import os

# import pandas as pd
# from datasets import Dataset


# def load_dataset(add_caption = True) -> Dataset:
#     """Load dataset."""
    
#     dataset_pandas = pd.read_csv(CSV_FILE)[['text_corrected','offensive']]
    
#     dataset_pandas['offensive'] = np.where(dataset_pandas['offensive'] == 'not_offensive', 'not offensive', 'offensive')

    
#     dataset_pandas = dataset_pandas.rename(columns={'offensive': 'label','text_corrected' : 'text'})

#     dataset_pandas["label"] = dataset_pandas["label"].astype(str)
#     dataset_pandas["text"] = dataset_pandas["text"].astype(str)
    
#     dataset = Dataset.from_pandas(dataset_pandas)
#     dataset = dataset.shuffle(seed=42)
#     dataset = dataset.train_test_split(test_size=0.2,seed=42,stratify_by_column='label')

#     return dataset

In [10]:
# def downsample_training_data(dataset: Dataset) -> Dataset:
#     """Downsample the training data to have equal number of offensive and not offensive labels."""
    
#     train_dataset = dataset['train'].to_pandas()
    
#     # Separate the offensive and not offensive samples
#     offensive_samples = train_dataset[train_dataset['label'] == 'offensive']
#     not_offensive_samples = train_dataset[train_dataset['label'] == 'not offensive']
    
#     # Determine the smaller class size
#     min_samples = min(len(offensive_samples), len(not_offensive_samples))
    
#     # Downsample the larger class
#     offensive_samples_downsampled = offensive_samples.sample(min_samples, random_state=42)
#     not_offensive_samples_downsampled = not_offensive_samples.sample(min_samples, random_state=42)
    
#     # Combine the downsampled samples
#     train_dataset_downsampled = pd.concat([offensive_samples_downsampled, not_offensive_samples_downsampled])
    
#     # Shuffle the combined dataset
#     train_dataset_downsampled = train_dataset_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)
    
#     # Convert back to Dataset format
#     train_dataset_downsampled = Dataset.from_pandas(train_dataset_downsampled)
    
#     # Update the train split with the downsampled dataset
#     dataset['train'] = train_dataset_downsampled
    
#     return dataset

In [11]:
# # Load the dataset
# #dataset = load_dataset()

# # Print the number of examples per class for both train and test splits
# train_labels = dataset['train']['label']
# val_labels = dataset['validation']['label']
# test_labels = dataset['test']['label']

# train_counts = pd.Series(train_labels).value_counts()
# val_counts = pd.Series(val_labels).value_counts()
# test_counts = pd.Series(test_labels).value_counts()

# print("Number of examples per class in the training set:")
# print(train_counts)
# print("\nNumber of examples per class in the val set:")
# print(val_counts)
# print("\nNumber of examples per class in the test set:")
# print(test_counts)

In [12]:
# # Downsample the training data
# dataset = downsample_training_data(dataset)

# dataset['train'][1]

In [13]:
# train_labels = dataset['train']['label']
# test_labels = dataset['test']['label']

# train_counts = pd.Series(train_labels).value_counts()
# test_counts = pd.Series(test_labels).value_counts()

# print("Number of examples per class in the training set:")
# print(train_counts)
# print("\nNumber of examples per class in the test set:")
# print(test_counts)

In [14]:
# Load tokenizer of FLAN-t5
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [15]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.

tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"],dataset["test"]]).map(
    lambda x: tokenizer(x["text"], truncation=True),
    batched=True,
    remove_columns=["text", "label"],
)

Map:   0%|          | 0/5859 [00:00<?, ? examples/s]

In [16]:
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

Max source length: 512


In [17]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["label"], truncation=True),
    batched=True,
    remove_columns=["text", "label"],
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/5300 [00:00<?, ? examples/s]

Max target length: 512


In [18]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir='./results"',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    fp16=False,
    seed = 110,
    learning_rate=LR,
    num_train_epochs=epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    generation_max_length = MAX_LEN
)

In [19]:
def preprocess_function(sample: Dataset, padding: str = "max_length") -> dict:
    """Preprocess the dataset."""

    # add prefix to the input for t5
    #prefix = 'Is the following text offensive or not offensive ? :'
    inputs = [item for item in sample["text"]]
    # tokenize inputs
    model_inputs = tokenizer(
        inputs, max_length=max_source_length, padding=padding, truncation=True
    )

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        text_target=sample["label"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(la if la != tokenizer.pad_token_id else -100) for la in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
def postprocess_text(
    preds: List[str], labels: List[str]
) -> Tuple[List[str], List[str]]:
    """helper function to postprocess text"""
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

In [21]:
tokenizer.vocab_size

32100

In [22]:
from datasets import load_metric
import numpy as np

# Load the metrics
bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Ensure that all token IDs are within the valid range
    valid_token_ids = range(tokenizer.vocab_size)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = []
    for pred in preds:
        try:
            #pred = [token_id for token_id in pred if token_id in valid_token_ids]
            decoded_preds.append(tokenizer.decode(pred, skip_special_tokens=True))
        except Exception as e:
            print(f"Error decoding prediction: {e}")
            for token_id in pred:
                print(token_id)
            decoded_preds.append("")
    
    #decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    
    # Compute ROUGE score
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rouge1", "rouge2", "rougeL"])

    result = {
        "bleu": round(bleu["score"], 4),
        "rouge1": round(rouge["rouge1"].mid.fmeasure * 100, 4),
        "rouge2": round(rouge["rouge2"].mid.fmeasure * 100, 4),
        "rougeL": round(rouge["rougeL"].mid.fmeasure * 100, 4),
    }
    
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    
    return result

/tmp/ipykernel_23/2137121049.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     print(preds)   
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
#     #print(decoded_preds)
    
#     decoded_preds = [1 if pred == "offensive" else 0 for pred in decoded_preds]
#     decoded_labels = [1 if label == "offensive" else 0 for label in decoded_labels]

#     # Compute precision, recall, and F1 score for both classes and macro average
#     precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average=None, labels=[0, 1])
#     precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='macro')
#     acc = accuracy_score(decoded_labels,decoded_preds)
    
#     result = {
#         "accuracy" : round(acc * 100, 4),
#         "precision_not_offensive": round(precision[0] * 100, 4),
#         "recall_not_offensive": round(recall[0] * 100, 4),
#         "f1_not_offensive": round(f1[0] * 100, 4),
#         "precision_offensive": round(precision[1] * 100, 4),
#         "recall_offensive": round(recall[1] * 100, 4),
#         "f1_offensive": round(f1[1] * 100, 4),
#         "precision_macro": round(precision_macro * 100, 4),
#         "recall_macro": round(recall_macro * 100, 4),
#         "f1_macro": round(f1_macro * 100, 4),
#     }
    
#     prediction_lens = [
#         np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
#     ]
#     result["gen_len"] = np.mean(prediction_lens)
    
#     return result


In [24]:
def train() -> None:
    """Train and test the model."""

    tokenized_dataset = dataset.map(
        preprocess_function, batched=True, remove_columns=['text','label']
    )
    
    print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

    # load model from the hub
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    nltk.download("punkt")

    # we want to ignore tokenizer pad token in the loss
    label_pad_token_id = -100
    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8,
    )

    # Create Trainer instance
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['validation'],
        compute_metrics=compute_metrics
    )
    

    # TRAIN
    trainer.train()
    
    # TEST
    #evaluation_results = trainer.evaluate(eval_dataset = tokenized_dataset['validation'])
    #print(evaluation_results)

In [25]:
train()

Map:   0%|          | 0/3902 [00:00<?, ? examples/s]

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/1398 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Gen Len
1,No log,2.154177,3.840400,23.149900,6.550100,17.760600,375.005367
2,2.196500,2.012401,6.470200,29.675800,9.802700,20.919900,313.550984
3,1.819500,1.947108,7.492000,30.833200,10.402400,21.408000,281.329159
4,1.706700,1.910053,8.854400,33.619000,11.720700,22.368300,247.973166
5,1.637600,1.883562,9.081500,33.755300,11.533500,22.495300,243.769231
6,1.588000,1.863442,9.087200,34.526000,11.838800,22.667200,223.973166
7,1.567300,1.851963,9.362000,34.853500,12.093800,22.932000,223.633274
8,1.543500,1.842345,9.283000,35.053800,12.147400,22.986300,219.391771
9,1.526100,1.838583,9.378000,35.027400,12.090100,22.871800,224.620751
10,1.517700,1.836529,9.315800,34.896300,11.977000,22.813400,226.364937
